In [ ]:
# 라이브러리 호출
import numpy as np
import pandas as pd
from datetime import datetime

In [ ]:
# 데이터 불러오기
rawData = pd.read_csv('TPSS_region_OD3.csv', encoding='cp949', sep=",")
rawData.head(30)

In [ ]:
# OB 와 DB 가 같은 지역 제거
# 날짜는 20230920 인 경우만으로 필터링
# population 이 적어도 100 이 넘는 지역만
# population 의 내림차순으로 정렬하기.

filterdDf =rawData[(rawData['OB']!=rawData['DB'])&(rawData['Date']==20230920)&(rawData['population']>100)].sort_values('population', ascending= False)

In [ ]:
filterdDf =filterdDf.loc[:,['OB','DB','population']]

In [ ]:
# A에서 B로 이동하는 데이터 복사
output1 = filterdDf.copy()

# B에서 A로 이동하는 데이터 복사
output2 = filterdDf.copy()
# 열 이름 바꾸기
output2.columns = ['DB', 'OB', 'population']

# 두 데이터프레임 병합
merged_df = pd.concat([output1, output2])

# # Location_1 및 Location_2 열의 값을 ID로 바꾸기
# merged_df['DB'] = merged_df['DB'].map(region_id_mapping)
# merged_df['OB'] = merged_df['OB'].map(region_id_mapping)

# 동일한 출발지와 도착지를 가진 데이터 합치기
result_df = merged_df.groupby(['DB', 'OB']).sum().reset_index()

# (Location_1=지역1 & Location_2=지역3)와 (Location_1=지역3 & Location_2=지역1)을 같은 것으로 처리
result_df['Unique_DB'] = result_df[['DB', 'OB']].min(axis=1)
result_df['Unique_OB'] = result_df[['DB', 'OB']].max(axis=1)

# 중복된 데이터 제거
result_df = result_df.drop_duplicates(subset=['Unique_DB', 'Unique_OB'])

# 불필요한 열 제거
result_df = result_df.drop(columns=['Unique_DB', 'Unique_OB'])
